In this notebook Graph Nets library is used to learn to predict the motion of a set of masses conncted by springs.

The network is trained to predict the behaviour of a chain of five masses, connected by identical springs. The first and last masses are fixed; the others are subject to gravity.

In [10]:
import time

from graph_nets import blocks
from graph_nets import utils_tf
from graph_nets.demos_tf2 import models
from matplotlib import pyplot as plt
import numpy as np
import sonnet as snt
import tensorflow as tf
import torch

import seaborn as sns

In [5]:
def base_graph(n, d):
  """Define a basic mass-spring system graph structure.

  These are n masses (1kg) connected by springs in a chain-like structure. The
  first and last masses are fixed. The masses are vertically aligned at the
  start and are d meters apart; this is also the rest length for the springs
  connecting them. Springs have spring constant 50 N/m and gravity is 10 N in
  the negative y-direction.

  Args:
    n: number of masses
    d: distance between masses (as well as springs' rest length)

  Returns:
    data_dict: dictionary with globals, nodes, edges, receivers and senders
        to represent a structure like the one above.
  """
  # Nodes
  # Generate initial position and velocity for all masses.
  # The left-most mass has is at position (0, 0); other masses (ordered left to
  # right) have x-coordinate d meters apart from their left neighbor, and
  # y-coordinate 0. All masses have initial velocity 0m/s.
  nodes = np.zeros((n, 5), dtype=np.float32)
  half_width = d * n / 2.0
  nodes[:, 0] = np.linspace(
      -half_width, half_width, num=n, endpoint=False, dtype=np.float32)
  # indicate that the first and last masses are fixed
  nodes[(0, -1), -1] = 1.

  # Edges.
  edges, senders, receivers = [], [], []
  for i in range(n - 1):
    left_node = i
    right_node = i + 1
    # The 'if' statements prevent incoming edges to fixed ends of the string.
    if right_node < n - 1:
      # Left incoming edge.
      edges.append([50., d])  #edges have 2 features, spring constant and spring length
      senders.append(left_node)
      receivers.append(right_node)
    if left_node > 0:
      # Right incoming edge.
      edges.append([50., d])
      senders.append(right_node)
      receivers.append(left_node)

  return {
      "globals": [0., -10.],
      "nodes": nodes,
      "edges": edges,
      "receivers": receivers,
      "senders": senders
  }

Each node has 5 features:
* x
* y
* v_x
* v_y
* is_fixed

In [7]:
base_graph=base_graph(10,2)
base_graph
#first and last mass send and edge, don't receive any edge

{'globals': [0.0, -10.0],
 'nodes': array([[-10.,   0.,   0.,   0.,   1.],
        [ -8.,   0.,   0.,   0.,   0.],
        [ -6.,   0.,   0.,   0.,   0.],
        [ -4.,   0.,   0.,   0.,   0.],
        [ -2.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.],
        [  2.,   0.,   0.,   0.,   0.],
        [  4.,   0.,   0.,   0.,   0.],
        [  6.,   0.,   0.,   0.,   0.],
        [  8.,   0.,   0.,   0.,   1.]], dtype=float32),
 'edges': [[50.0, 2],
  [50.0, 2],
  [50.0, 2],
  [50.0, 2],
  [50.0, 2],
  [50.0, 2],
  [50.0, 2],
  [50.0, 2],
  [50.0, 2],
  [50.0, 2],
  [50.0, 2],
  [50.0, 2],
  [50.0, 2],
  [50.0, 2],
  [50.0, 2],
  [50.0, 2]],
 'receivers': [1, 2, 1, 3, 2, 4, 3, 5, 4, 6, 5, 7, 6, 8, 7, 8],
 'senders': [0, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 9]}

In [8]:
def hookes_law(receiver_nodes, sender_nodes, k, x_rest):
  """Applies Hooke's law to springs connecting some nodes.

  Args:
    receiver_nodes: Ex5 tf.Tensor of [x, y, v_x, v_y, is_fixed] features for the
      receiver node of each edge.
    sender_nodes: Ex5 tf.Tensor of [x, y, v_x, v_y, is_fixed] features for the
      sender node of each edge.
    k: Spring constant for each edge.
    x_rest: Rest length of each edge.

  Returns:
    Nx2 Tensor of the force [f_x, f_y] acting on each edge.
  """
  #compute the coordinates difference between receivers and senders
  diff = receiver_nodes[..., 0:2] - sender_nodes[..., 0:2]
  #compute the distance between receivers and senders
  x = tf.norm(diff, axis=-1, keepdims=True)
  
  force_magnitude = -1 * tf.multiply(k, (x - x_rest) / x)
  force = force_magnitude * diff
  return force

In [ ]:
def euler_integration(nodes, force_per_node, step_size):
  """Applies one step of Euler integration.

  Args:
    nodes: Ex5 tf.Tensor of [x, y, v_x, v_y, is_fixed] features for each node.
    force_per_node: Ex2 tf.Tensor of the force [f_x, f_y] acting on each edge.
    step_size: Scalar.

  Returns:
    A tf.Tensor of the same shape as `nodes` but with positions and velocities
        updated.
  """
  is_fixed = nodes[..., 4:5]
  # set forces to zero for fixed nodes
  force_per_node *= 1 - is_fixed
  new_vel = nodes[..., 2:4] + force_per_node * step_size
  return new_vel